In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('fnc_train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.shape

(20800, 5)

In [5]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
df = df.dropna()

In [8]:
X = df.drop('label', axis=1)
y = df['label']

In [9]:
X.shape, y.shape

((18285, 4), (18285,))

In [13]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, Dense, LSTM, Dropout, Bidirectional
from tensorflow.keras.layers import Bidirectional

In [14]:
messages = X.copy()
messages['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [19]:
messages.reset_index(inplace=True)

In [20]:
messages.head()

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [15]:
# finding the root words of all the words in messages by stemming

In [21]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [18]:
# nltk.download('stopwords')

In [27]:
corpus = []
ps = PorterStemmer()
for i in range(0,len(messages)):
    roots = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    roots = roots.lower()
    roots = roots.split()

    roots = [ps.stem(word) for word in roots if not word in stopwords.words('english')]
    roots = ' '.join(roots)
    corpus.append(roots)

In [30]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [32]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [33]:
voc_size = 5000

In [35]:
one_hot_rep = [one_hot(word, voc_size) for word in corpus]

In [38]:
one_hot_rep[0:4]

[[1035, 3915, 4589, 4214, 1503, 4104, 3582, 3244, 4680, 1896],
 [870, 1879, 1167, 1281, 1851, 3, 3625],
 [4381, 957, 2969, 3839],
 [4741, 1997, 4301, 3891, 2762, 1689]]

In [39]:
sent_len = 20
embedded_docs = pad_sequences(one_hot_rep, padding='pre', maxlen=sent_len)

In [44]:
embedded_docs.shape

(18285, 20)

In [87]:
embedded_vector_features = 40
model = Sequential()
model.add(Input(shape=(sent_len,)))
model.add(Embedding(voc_size, embedded_vector_features))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [88]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ (None, 20, 40)              │         200,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_3 (Bidirectional)      │ (None, 200)                 │         112,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             201 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 313,001 (1.19 MB)

 Trainable params: 313,001 (1.19 MB)

 Non-trainable params: 0 (0.00 B)

In [89]:
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [90]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42) 

In [91]:
X_train.shape, X_test.shape

((12250, 20), (6035, 20))

In [92]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 27s 74ms/step - accuracy: 0.7951 - loss: 0.3976 - val_accuracy: 0.9185 - val_loss: 0.1965
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 12s 64ms/step - accuracy: 0.9493 - loss: 0.1375 - val_accuracy: 0.9198 - val_loss: 0.2013
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 13s 65ms/step - accuracy: 0.9708 - loss: 0.0848 - val_accuracy: 0.9133 - val_loss: 0.2442
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 12s 63ms/step - accuracy: 0.9799 - loss: 0.0628 - val_accuracy: 0.9123 - val_loss: 0.2631
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 12s 64ms/step - accuracy: 0.9867 - loss: 0.0448 - val_accuracy: 0.9104 - val_loss: 0.2840
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 13s 65ms/step - accuracy: 0.9912 - loss: 0.0311 - val_accuracy: 0.9075 - val_loss: 0.3391
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 12s 63ms/step - accuracy: 0.9928 - loss: 0.0240 - val_accuracy: 0.9067 - val_loss: 0.3286
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 12s 63ms/step - accuracy: 0.9943 - loss: 0.0190 - 

In [93]:
# Dropout 

In [94]:
y_pred = model.predict(X_test)

189/189 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step


In [95]:
y_pred = (y_pred > 0.5).astype('int')

In [96]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [97]:
confusion_matrix(y_test, y_pred)

array([[3109,  310],
       [ 249, 2367]])

In [98]:
accuracy_score(y_test, y_pred)

0.9073736536868269